<a href="https://colab.research.google.com/github/Rakshithts/myprojects/blob/main/fine_tuned_model_for_news_format_sentence_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets peft accelerate -q

In [ ]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
import os
import torch


In [ ]:
# Disable W&B logging
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
# 1. Load a larger dataset (AG News)
raw_dataset = load_dataset("ag_news")
train_texts = raw_dataset["train"]["text"][:20000]

dataset = Dataset.from_dict({"text": train_texts})

In [ ]:
# 2. Load tokenizer & model
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
# 3. Tokenize with labels
def tokenize(batch):
    tokenized = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=64)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_ds = dataset.map(tokenize, batched=True, remove_columns=["text"])

In [ ]:
# 4. Apply LoRA (PEFT)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],  # GPT-2 attention layers
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
# 5. Training setup
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,   # bigger batch size for speed
    learning_rate=5e-4,
    num_train_epochs=5,
    logging_steps=50,
    report_to="none",
    save_strategy="no",
    fp16=torch.cuda.is_available()   # mixed precision if GPU supports it
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds
)


In [ ]:
# 6. Train
trainer.train()

In [ ]:
# 7. Test generation
prompt = "The weather today"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=30, do_sample=True, top_k=40, top_p=0.9)
print("\nGenerated text:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
# 7. Test generation
prompt = "The movie kept the audience"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=30, do_sample=True, top_k=40, top_p=0.9)
print("\nGenerated text:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
from transformers import AutoModelForCausalLM

# Function for generating text
def generate_text(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=30,
        do_sample=True,
        top_k=40,
        top_p=0.9
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "The politics"

# 1. Load base DistilGPT-2 for BEFORE result
base_model = AutoModelForCausalLM.from_pretrained("distilgpt2").to(model.device)
base_model.eval()

print("=== BEFORE Fine-Tuning ===")
before_text = generate_text(base_model, tokenizer, prompt)
print(before_text)

# 2. AFTER: Fine-tuned model (already trained above)
print("\n=== AFTER Fine-Tuning ===")
after_text = generate_text(model, tokenizer, prompt)
print(after_text)
